# File Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import os
import json
import datetime
import pysmile
import pysmile_license
import sys
import json
from colormap import rgb2hex
from collections import ChainMap
import re
from scipy.stats import lognorm
import math

# Load Data

In [3]:

base_dir = r"C:\Users\shannonb\Documents\Pacific_coastal_modelling\Work_extracted\Tuvalu_shoreline_change"

df_precip = pd.read_csv(base_dir+'\Preprocessed_datasets/precipitation.csv')
df_tides_nanumaga = pd.read_csv(base_dir+'\Preprocessed_datasets/nanumaga_time_with_tide.csv',index_col=[0]).drop('time',axis=1).reset_index().rename(columns={'index':'time'})
df_tides_nanumea = pd.read_csv(base_dir+'\Preprocessed_datasets/nanumea_time_with_tide.csv',index_col=[0]).drop('time',axis=1).reset_index().rename(columns={'index':'time'})

df_tides_nanumea['time'] = [x.split(' ')[0] for x in list(df_tides_nanumea.time)]
df_tides_nanumaga['time'] = [x.split(' ')[0] for x in list(df_tides_nanumaga.time)]

def convt_str_datetime(df):
    df['time'] = \
        [datetime.datetime(int(x.split('-')[0]),
                           int(x.split('-')[1]),
                           int(x.split('-')[2])) for x in df.time]
    return(df)

df_precip = convt_str_datetime(df_precip)
df_tides_nanumaga = convt_str_datetime(df_tides_nanumaga)
df_tides_nanumea = convt_str_datetime(df_tides_nanumea)

df_funafuti_tides = pd.read_csv(base_dir+'\Preprocessed_datasets\h025.csv')
df_funafuti_tides.index = [datetime.datetime(y,m,d) for y,m,d in zip(df_funafuti_tides.year,df_funafuti_tides.month,df_funafuti_tides.date)]



In [4]:
pd.read_csv(base_dir+r'\Preprocessed_datasets\Nanumea_dates.csv').date

0     03/07/2003
1     10/01/2005
2     10/07/2007
3     21/03/2009
4     16/08/2010
5     21/03/2011
6     11/12/2012
7     04/05/2013
8     05/11/2014
9     16/03/2015
10    26/11/2016
11    12/09/2017
12    26/08/2018
13    26/05/2019
14    26/08/2020
15    06/06/2021
Name: date, dtype: object

In [5]:
nanumanga_dates = list(pd.read_csv(base_dir+r'\Preprocessed_datasets\Nanumaga_dates.csv').date)
nanumanga_dates = \
    [datetime.datetime(int(x.split('/')[2]),int(x.split('/')[1]),int(x.split('/')[0])) for x in nanumanga_dates]


nanumea_dates = list(pd.read_csv(base_dir+r'\Preprocessed_datasets\Nanumea_dates.csv').date)
nanumea_dates = \
    [datetime.datetime(int(x.split('/')[2]),int(x.split('/')[1]),int(x.split('/')[0])) for x in nanumea_dates]

nanumanga_dates = {x.year:x for x in nanumanga_dates}
nanumea_dates = {x.year:x for x in nanumea_dates}

df_areas = pd.read_csv(base_dir+r'\Processed_data\areas.csv')
df_areas_nanumanga = df_areas.copy()
df_areas_nanumanga['year'] = [nanumanga_dates[year] if year in nanumanga_dates else np.nan for year in df_areas.year]
df_areas_nanumanga.dropna(inplace=True)
df_areas_nanumanga = df_areas_nanumanga[(df_areas_nanumanga.id==1)&(df_areas_nanumanga.islet=='Nanumanga')]
df_areas_nanumanga = df_areas_nanumanga.sort_values('proxy')

df_areas = pd.read_csv(base_dir+r'\Processed_data\areas.csv')
df_areas_nanumea = df_areas.copy()
df_areas_nanumea['year'] = [nanumea_dates[year] if year in nanumea_dates else np.nan for year in df_areas.year]
df_areas_nanumea.dropna(inplace=True)
df_areas_nanumea = df_areas_nanumea[(df_areas_nanumea.id==1)&(df_areas_nanumea.islet=='Nanumea')]
df_areas_nanumea = df_areas_nanumea.sort_values('proxy')

In [6]:
df_shoreline_segments = pd.read_csv('updated_shoreline_proxies_withpop.csv')
df_shoreline_segments['year2'] = [nanumea_dates[x] if x in list(nanumea_dates.keys()) else np.nan for x in df_shoreline_segments['year2']]
df_shoreline_segments.dropna(inplace=True)
df_shoreline_segments.rename(columns={'TOB':'BeachToe','VL':'VegetationLine','WM':'WaterMark','pop_binary':'PopulationPresent'},inplace=True)


In [7]:
df_shoreline_segments

,Unnamed: 0,x,y,year2,clust,BeachToe,VegetationLine,WaterMark,PopulationPresent
2,2,176.110309,-5.669372,2005-01-10,0.0,-0.000076,-0.000249,-0.007585,1
4,4,176.110309,-5.669372,2009-03-21,0.0,0.036856,-0.017534,-0.000219,1
5,5,176.110309,-5.669372,2010-08-16,0.0,-0.000107,-0.000295,-0.000220,1
6,6,176.110309,-5.669372,2011-03-21,0.0,-0.005512,-0.000277,-0.000242,1
7,7,176.110309,-5.669372,2012-12-11,0.0,-0.005514,-0.005160,-0.006985,1
...,...,...,...,...,...,...,...,...,...
3995,3995,176.136436,-5.701126,2016-11-26,2.0,0.001280,0.000000,0.000894,0
3996,3996,176.136436,-5.701126,2017-09-12,2.0,-0.000189,0.000000,-0.000007,0
3997,3997,176.136436,-5.701126,2018-08-26,2.0,0.000076,0.000000,0.000141,0
3998,3998,176.136436,-5.701126,2019-05-26,2.0,0.000043,0.000000,0.000033,0


In [8]:
# df_shoreline_segments = df_shoreline_segments.drop(['year2','x','y'],axis=1)
number_of_clusts = int(np.max(pd.read_csv('updated_shoreline_proxies_withpop.csv').clust)+1)

runs = 10000

df_rand_gen = pd.concat([pd.DataFrame(index=range(runs)).reset_index()]*len(np.unique(df_shoreline_segments.year2)))
df_rand_gen['year2'] = list(np.unique(df_shoreline_segments.year2))*runs


for i in range(number_of_clusts):
    df_clust = df_shoreline_segments[df_shoreline_segments.clust==i]
    
    df_rand_gen_year = pd.DataFrame()
    
    for year,group in df_clust.groupby('year2'):

        mean = np.mean(group[['BeachToe','VegetationLine','WaterMark','PopulationPresent']],axis=0)
        stdev = np.std(group[['BeachToe','VegetationLine','WaterMark','PopulationPresent']])

        df_rand_gen_year_temp = pd.DataFrame({x:np.random.normal(loc=mean,scale=stdev) for x in range(1000)}).T.rename(columns={
                    0:f'BeachToe{i}',
                    1:f'VegetationLine{i}',
                    2:f'WaterMark{i}',
                    3:f'PopulationPresent{i}'
                })
        
        df_rand_gen_year_temp['year2'] = year
        
        df_rand_gen_year = pd.concat([df_rand_gen_year_temp,df_rand_gen_year])
        
        
    df_rand_gen_year = df_rand_gen_year.reset_index()
        
    df_rand_gen = pd.merge(df_rand_gen,df_rand_gen_year,on=['year2','index'],how='outer')

# Normalise them
for x in [y for y in list(df_rand_gen) if y not in ['index','year2']]:
    df_rand_gen[x] = df_rand_gen[x]-np.mean(df_rand_gen[x])

In [9]:
# Normalise them
for x in [y for y in list(df_rand_gen) if y not in ['index','year2']]:
    print('errosion',len(df_rand_gen[df_rand_gen[x]<0]))
    print('accretion',len(df_rand_gen[df_rand_gen[x]>0]))
    print('')

errosion 6495
accretion 6505

errosion 6580
accretion 6420

errosion 6283
accretion 6717

errosion 6454
accretion 6546

errosion 5920
accretion 7080

errosion 6601
accretion 6399

errosion 6617
accretion 6383

errosion 6482
accretion 6518

errosion 8316
accretion 4684

errosion 6586
accretion 6414

errosion 6453
accretion 6547

errosion 6475
accretion 6525

errosion 5278
accretion 7722

errosion 5729
accretion 7271

errosion 6286
accretion 6714

errosion 0
accretion 0



In [10]:
df_shoreline_segments = df_rand_gen.copy()

In [11]:
np.mean(df_shoreline_segments)

C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


index                 4.999500e+03
BeachToe0             2.989062e-20
VegetationLine0      -1.601283e-20
WaterMark0            2.498002e-19
PopulationPresent0    2.743105e-17
BeachToe1             2.028292e-20
VegetationLine1      -5.310923e-20
WaterMark1            1.067522e-21
PopulationPresent1   -1.831014e-17
BeachToe2             1.011477e-19
VegetationLine2      -5.017354e-20
WaterMark2           -1.777424e-19
PopulationPresent2   -2.596214e-18
BeachToe3            -1.305046e-19
VegetationLine3      -7.526031e-20
WaterMark3           -4.270089e-20
PopulationPresent3    0.000000e+00
dtype: float64

In [12]:
times = np.unique(df_shoreline_segments.year2)

In [13]:
with open(base_dir+r'\Processed_data\{}.json'.format('Nanumaga'), 'r') as f:
    total_wave_power_nanumanga = json.load(f)
    
with open(base_dir+r'\Processed_data\{}.json'.format('Nanumea'), 'r') as f:
    total_wave_power_nanumea = json.load(f)
    
event_sizes_nanumanga = list(total_wave_power_nanumanga[list(total_wave_power_nanumanga.keys())[0]]['events'].keys())
event_sizes_nanumea = list(total_wave_power_nanumea[list(total_wave_power_nanumea.keys())[0]]['events'].keys())

In [14]:
df_nanumea_mean_wave = pd.DataFrame.from_dict({key:item['daily_mean'] for key,item in total_wave_power_nanumea.items()},orient='index')
time_frames = list(df_nanumea_mean_wave.index)

df_nanumea_mean_wave['year1'] = [datetime.datetime(int(x.split('), ')[0].split(', ')[0].replace('(datetime.datetime(','')),
                   int(x.split('), ')[0].split(', ')[1]),
                   int(x.split('), ')[0].split(', ')[2])) for x in time_frames]

df_nanumea_mean_wave['year2'] = [datetime.datetime(int(x.split('), ')[1].split(', ')[0].replace('datetime.datetime(','')),
                   int(x.split('), ')[1].split(', ')[1]),
                   int(x.split('), ')[1].split(', ')[2])) for x in time_frames]
df_nanumea_mean_wave.reset_index(inplace=True,drop=True)
df_nanumea_mean_wave.rename(columns={0:'AverageDailyWavePower'},inplace=True)

df_nanumea_mean_wave

,AverageDailyWavePower,year1,year2
0,122859.220620,2003-07-03,2005-01-10
1,144623.222185,2005-01-10,2007-07-10
2,181866.566465,2007-07-10,2009-03-21
3,209945.810347,2009-03-21,2010-08-16
4,133124.039582,2010-08-16,2011-03-21
5,225004.446799,2011-03-21,2012-12-11
6,269176.738938,2012-12-11,2013-05-04
7,186955.163976,2013-05-04,2014-11-05
8,173571.954346,2014-11-05,2015-03-16
9,145893.827616,2015-03-16,2016-11-26


## Sort the tidal data



# Setup a new network

In [15]:
net = pysmile.Network()

all_nodes = ['Tide','MeanSeaLevel','AverageDailyWavePower','Precipitation','BeachToe','VegetationLine','WaterMark'] # TideGrad
proxy_nodes = ['BeachToe','VegetationLine','WaterMark']

for node in [x for x in all_nodes if x not in proxy_nodes]:
    net.add_node(pysmile.NodeType.CPT,node)
        
for node in [x for x in all_nodes if x not in proxy_nodes]:
    if node!='Tide':
        for outcome in ['Low','Medium','High']:
            net.add_outcome(node,outcome)
    else:
        for outcome in ['Low','High']:
            net.add_outcome(node,outcome)

In [16]:
# Get the number of clusters

number_of_clusts = int(np.max(pd.read_csv('updated_shoreline_proxies.csv').clust)+1)

proxy_nodes_per_clust_dict = {x:[pn+str(x) for pn in proxy_nodes] for x in range(number_of_clusts)}

for x in range(number_of_clusts):
    
    for node in proxy_nodes+['PopulationPresent']:
        net.add_node(pysmile.NodeType.CPT,f'{node}{x}')
    
    for node in [f'BeachToe{x}',f'WaterMark{x}']:
        for outcome in ['Erosion','Accretion']:
            net.add_outcome(node,outcome)

    for node in [f'VegetationLine{x}']:
        for outcome in ['Advancement','Recession']:
            net.add_outcome(node,outcome)
            
    for node in [f'PopulationPresent{x}']:
        for outcome in ['Present','NotPresent']:
            net.add_outcome(node,outcome)

    net.add_arc(f'Tide',f'WaterMark{x}')
    net.add_arc(f'MeanSeaLevel',f'WaterMark{x}')
    net.add_arc(f'AverageDailyWavePower',f'WaterMark{x}')
    net.add_arc(f'MeanSeaLevel',f'BeachToe{x}')
    net.add_arc(f'AverageDailyWavePower',f'BeachToe{x}')
    net.add_arc(f'BeachToe{x}',f'VegetationLine{x}')
    net.add_arc(f'Precipitation',f'VegetationLine{x}')
    net.add_arc(f'PopulationPresent{x}',f'VegetationLine{x}')
    net.add_arc(f'PopulationPresent{x}',f'BeachToe{x}')
    net.add_arc(f'PopulationPresent{x}',f'WaterMark{x}')
    
# Try adding in some arcs between the clusters
# for x in range(number_of_clusts):
#     for y in range(number_of_clusts):
#         if (x!=y)&(x<y):
#             net.add_arc(f'BeachToe{x}',f'BeachToe{y}')


In [17]:
net.get_all_node_ids()

['Tide',
 'MeanSeaLevel',
 'AverageDailyWavePower',
 'Precipitation',
 'BeachToe0',
 'VegetationLine0',
 'WaterMark0',
 'PopulationPresent0',
 'BeachToe1',
 'VegetationLine1',
 'WaterMark1',
 'PopulationPresent1',
 'BeachToe2',
 'VegetationLine2',
 'WaterMark2',
 'PopulationPresent2',
 'BeachToe3',
 'VegetationLine3',
 'WaterMark3',
 'PopulationPresent3']

In [18]:
        
# Delete the two default states
for node in net.get_all_node_ids():
    net.delete_outcome(node,'State0')
    net.delete_outcome(node,'State1')
    
    
net.set_node_position('Tide',800,50,50,50)
net.set_node_position('MeanSeaLevel',600,50,50,50)
net.set_node_position('AverageDailyWavePower',400,50,50,50)
net.set_node_position('Precipitation',600,600,50,50)

for x in range(number_of_clusts):
    net.set_node_position(f'BeachToe{x}',400*x+50,200,50,50)
    net.set_node_position(f'WaterMark{x}',400*x+250,200,50,50)
    net.set_node_position(f'VegetationLine{x}',400*x+150,350,50,50)    


for node in net.get_all_nodes():
    net.set_node_chart_enabled(node,1)

In [19]:
# df_nanumea_proxies = df_areas_nanumea.drop(['islet','id'],axis=1).set_index('year').pivot(columns='proxy')

In [20]:
# df_nanumea_proxies

In [21]:
df_shoreline_segments = df_shoreline_segments[df_shoreline_segments.year2!=np.min(df_shoreline_segments.year2)]

In [22]:
df_shoreline_segments = df_shoreline_segments.set_index('year2')
df_tides_nanumea = df_tides_nanumea.set_index('time')

In [23]:
df_nanumea_mean_wave = df_nanumea_mean_wave.set_index('year2')

In [24]:
df_shoreline_segments = df_shoreline_segments.join(df_tides_nanumea[['level','grad']],how='left')

In [25]:
df_shoreline_segments = df_shoreline_segments.join(df_nanumea_mean_wave['AverageDailyWavePower'],how='left')

In [26]:
df_shoreline_segments = df_shoreline_segments.rename(columns={'level':'Tide','grad':'TideGrad'})

In [27]:
df_shoreline_segments

,index,BeachToe0,VegetationLine0,WaterMark0,PopulationPresent0,BeachToe1,VegetationLine1,WaterMark1,PopulationPresent1,BeachToe2,VegetationLine2,WaterMark2,PopulationPresent2,BeachToe3,VegetationLine3,WaterMark3,PopulationPresent3,Tide,TideGrad,AverageDailyWavePower
2009-03-21,1,-0.000005,0.002860,-0.000388,0.609756,0.000447,0.000971,0.000248,0.501557,-0.000652,-0.001058,0.002777,0.415467,0.000669,-0.004218,-0.001021,0.0,1864.0,189.5,181866.566465
2009-03-21,14,-0.003150,-0.002423,-0.002601,0.245082,0.000447,-0.000550,0.002182,0.399681,-0.000652,-0.002511,-0.000276,-0.300114,0.000669,0.003204,0.000616,0.0,1864.0,189.5,181866.566465
2009-03-21,27,0.008415,-0.004260,-0.001337,-0.306156,0.000447,-0.001617,-0.000730,-0.376403,-0.000652,0.002356,-0.008595,0.273130,0.000669,-0.001904,-0.001050,0.0,1864.0,189.5,181866.566465
2009-03-21,40,0.007083,0.008114,-0.003121,-0.721230,0.000447,0.000422,0.003058,-0.021867,-0.000652,0.001752,-0.010202,0.208264,0.000669,-0.003836,-0.000025,0.0,1864.0,189.5,181866.566465
2009-03-21,53,0.002420,-0.002379,0.000791,-0.143358,0.000447,-0.000322,0.001293,0.056488,-0.000652,-0.000366,-0.003248,0.771884,0.000669,-0.000489,-0.001064,0.0,1864.0,189.5,181866.566465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-26,9947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88835.567964
2020-08-26,9960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88835.567964
2020-08-26,9973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88835.567964
2020-08-26,9986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88835.567964


In [28]:

mean_precip_dict = {}

for year1,year2 in zip(times[:-1],times[1:]):
    
    mean_precip_dict.update({year2:{
        'MeanSeaLevel':np.mean(df_funafuti_tides[(df_funafuti_tides.index>year1)&(df_funafuti_tides.index<year2)&(df_funafuti_tides.level>0)].level),
        'Precipitation':np.mean(df_precip[(df_precip.time>year1)&(df_precip.time<year2)&(df_precip.precip>0)].precip)
    }
    })

df_mean_precip = pd.DataFrame.from_dict(mean_precip_dict,orient='index')

df_shoreline_segments = df_shoreline_segments.join(pd.DataFrame(mean_precip_dict).T,how='left')



In [29]:
# df_nanumea_proxies = df_areas_nanumea.drop(['islet','id'],axis=1).set_index('year')
# df_nanumea_TOB = df_nanumea_proxies[df_nanumea_proxies.proxy=='TOB'].drop('proxy',axis=1)
# df_nanumea_WM = df_nanumea_proxies[df_nanumea_proxies.proxy=='WM'].drop('proxy',axis=1)
# df_nanumea_VL = df_nanumea_proxies[df_nanumea_proxies.proxy=='VL'].drop('proxy',axis=1)

# df_nanumea_VL.columns = ['VegetationLine']
# df_nanumea_TOB.columns = ['BeachToe']
# df_nanumea_WM.columns = ['WaterMark']

In [30]:
# df_nanumea_VL.join(df_nanumea_TOB).join(df_nanumea_WM)


In [31]:
# df_areas_nanumea.drop(['islet','id'],axis=1).set_index('year').pivot(columns='proxy')

In [32]:
df_shoreline_segments = df_shoreline_segments.dropna()
df_shoreline_segments = df_shoreline_segments.reset_index()

nans = df_shoreline_segments.isna()

In [33]:
# var = 'MeanSeaLevel'
# df = df_shoreline_segments.copy()

# df = df.sort_values(var).reset_index(drop=True)



In [34]:
# df_nanumea_proxies = df_mean_precip.dropna().reset_index(drop=True)

df_mean_precip = df_shoreline_segments.copy()

for x in range(number_of_clusts):
    df_mean_precip[f'BeachToe{x}'] = ['Accretion' if x>=0 else 'Erosion' for x in df_mean_precip[f'BeachToe{x}']]
    df_mean_precip[f'VegetationLine{x}'] = ['Advancement' if x>=0 else 'Recession' for x in df_mean_precip[f'VegetationLine{x}']]
    df_mean_precip[f'WaterMark{x}'] = ['Accretion' if x>=0 else 'Erosion' for x in df_mean_precip[f'WaterMark{x}']]
#     df_mean_precip[f'TideGrad{x}'] = ['IncomingTide' if x>0 else 'OutgoingTide' for x in df_mean_precip.TideGrad]

def create_bins(df,var,states):

#     normal = ((df[var])-np.min(df[var]))/(np.max(df[var])-np.min(df[var]))
    if states==3:
        # This is for bins with bins that are the same size
#         normal[normal<=0.33333333] = -1
#         normal[(normal>=0.33333333)&(normal<=0.6666666)] = -2
#         normal[(normal>=0.6666666)] = -3
        # This is for bins that have the same number of points in each
        df = df.sort_values(var).reset_index(drop=True)
        df.loc[0:int(len(df)*0.333),var] = 'Low'
        df.loc[int(len(df)*0.333):int(len(df)*0.666),var] = 'Medium'
        df.loc[int(len(df)*0.666):,var] = 'High'
    
    
#         normal = normal.replace(-1,'Low')
#         normal = normal.replace(-2,'Medium')
#         normal = normal.replace(-3,'High')
    if states==2:
#         normal[normal<=0.5] = -1
#         normal[(normal>0.5)] = -3
#         normal = normal.replace(-1,'Low')
#         normal = normal.replace(-3,'High')
        df = df.sort_values(var).reset_index(drop=True)
        df.loc[0:int(len(df)*0.5),var] = 'Low'
        df.loc[int(len(df)*0.5):,var] = 'High'
        
#     df[var] = normal
    
    return(df)

df_mean_precip = create_bins(df_mean_precip,'Precipitation',3)
df_mean_precip = create_bins(df_mean_precip,'Tide',2)
df_mean_precip = create_bins(df_mean_precip,'AverageDailyWavePower',3)
df_mean_precip = create_bins(df_mean_precip,'MeanSeaLevel',3)

df_mean_precip.drop(['level_0','TideGrad'],axis=1,inplace=True)
df_mean_precip.dropna(inplace=True)
# df_mean_precip = pd.concat([df_mean_precip]*4)

# Make population node binary
for s in np.arange(0,4,1):
    df_mean_precip.loc[:,f'PopulationPresent{s}'] = ['NotPresent' if x<0.5 else 'Present' for x in df_mean_precip[f'PopulationPresent{s}']]

df_mean_precip.to_csv('dataset.csv',index=False)
df_mean_precip

,index,BeachToe0,VegetationLine0,WaterMark0,PopulationPresent0,BeachToe1,VegetationLine1,WaterMark1,PopulationPresent1,BeachToe2,...,WaterMark2,PopulationPresent2,BeachToe3,VegetationLine3,WaterMark3,PopulationPresent3,Tide,AverageDailyWavePower,MeanSeaLevel,Precipitation
0,289,Accretion,Recession,Erosion,NotPresent,Erosion,Recession,Erosion,NotPresent,Erosion,...,Accretion,NotPresent,Erosion,Advancement,Accretion,NotPresent,High,High,Low,Low
1,366,Accretion,Advancement,Erosion,NotPresent,Erosion,Advancement,Accretion,NotPresent,Erosion,...,Erosion,NotPresent,Erosion,Advancement,Erosion,NotPresent,High,High,Low,Low
2,947,Erosion,Recession,Erosion,NotPresent,Erosion,Recession,Accretion,NotPresent,Erosion,...,Accretion,NotPresent,Erosion,Recession,Erosion,NotPresent,High,High,Low,Low
3,106,Accretion,Recession,Erosion,NotPresent,Accretion,Recession,Accretion,NotPresent,Erosion,...,Accretion,NotPresent,Accretion,Advancement,Accretion,NotPresent,High,High,Low,Low
4,93,Accretion,Recession,Accretion,NotPresent,Accretion,Advancement,Accretion,NotPresent,Erosion,...,Erosion,NotPresent,Erosion,Advancement,Erosion,NotPresent,High,High,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,496,Erosion,Advancement,Accretion,Present,Erosion,Recession,Erosion,NotPresent,Erosion,...,Accretion,NotPresent,Accretion,Advancement,Accretion,NotPresent,High,Medium,High,Medium
8996,770,Erosion,Recession,Accretion,NotPresent,Accretion,Advancement,Erosion,NotPresent,Accretion,...,Erosion,NotPresent,Accretion,Advancement,Accretion,NotPresent,High,Medium,High,Medium
8997,608,Accretion,Advancement,Accretion,Present,Erosion,Advancement,Accretion,NotPresent,Accretion,...,Accretion,NotPresent,Accretion,Advancement,Accretion,NotPresent,High,Medium,High,Medium
8998,829,Erosion,Advancement,Accretion,NotPresent,Erosion,Recession,Erosion,NotPresent,Erosion,...,Accretion,NotPresent,Accretion,Recession,Accretion,NotPresent,High,Medium,High,Medium


In [35]:
# inc_nan_dict = {}
# for index,row in df_mean_precip.iterrows():
#     inc_nan_dict.update({
#         index:[x if y==False else '*' for col,x,y in zip(list(df_mean_precip),row,nans.loc[index])]
#     })

In [36]:
# df_for_training = pd.DataFrame.from_dict(inc_nan_dict).T
# df_for_training.columns = list(df_mean_precip)
# df_for_training.drop(['index','level_0','TideGrad'],axis=1,inplace=True)

# df_for_training.to_csv('dataset.csv',index=False)

In [37]:
ds_training = pysmile.learning.DataSet()
ds_training.read_file('dataset.csv')

# Looking at the dataframe

In [38]:
# fixed_nodes = [x for x in all_nodes if x not in list(df_nanumea_proxies.columns)]
# fixed_nodes

In [39]:
# net = pysmile.Network()

# all_nodes = ['Precipitation','BeachToe','VegetationLine']
# proxy_nodes = ['BeachToe','VegetationLine']

# for node in all_nodes:
#     net.add_node(pysmile.NodeType.CPT,node)
    
# for node in proxy_nodes:
#     for outcome in ['Erosion','Accretion']:
#         net.add_outcome(node,outcome)
        
# for node in [x for x in all_nodes if x not in proxy_nodes]:
#     for outcome in ['Low','Medium','High']:
#         net.add_outcome(node,outcome)
        

# net.add_arc('BeachToe','VegetationLine')
# net.add_arc('Precipitation','VegetationLine')

# # Delete the two default states
# for node in all_nodes:
#     net.delete_outcome(node,'State0')
#     net.delete_outcome(node,'State1')
    
# for node in ['Precipitation','BeachToe']:
#     net.set_node_definition(node,[1/net.get_outcome_count(node)]*net.get_outcome_count(node))

# for node in net.get_all_nodes():
#     net.set_node_chart_enabled(node,1)

# net.write_file("proxy_compare.xdsl")


In [40]:
em = pysmile.learning.EM()
em.learn(data=ds_training,net=net,matching=ds_training.match_network(net))

net.write_file("proxy_compare.xdsl")


In [41]:
ds_training

In [42]:
net.get_all_node_ids()

['Tide',
 'MeanSeaLevel',
 'AverageDailyWavePower',
 'Precipitation',
 'BeachToe0',
 'VegetationLine0',
 'WaterMark0',
 'PopulationPresent0',
 'BeachToe1',
 'VegetationLine1',
 'WaterMark1',
 'PopulationPresent1',
 'BeachToe2',
 'VegetationLine2',
 'WaterMark2',
 'PopulationPresent2',
 'BeachToe3',
 'VegetationLine3',
 'WaterMark3',
 'PopulationPresent3']

In [43]:
print('Accuracies')
for node in net.get_all_node_ids():
    validator = pysmile.learning.Validator(net,ds_training,ds_training.match_network(net))
    classNodehandle = net.get_node(node)
    validator.add_class_node(classNodehandle)
    validator.k_fold(em,5)
    acc = validator.get_accuracy(classNodehandle, 0)
    print(f'{node}:\t{round(acc,2)}')
    print('')

Accuracies
Tide:	0.78

MeanSeaLevel:	0.72

AverageDailyWavePower:	0.71

Precipitation:	0.48

BeachToe0:	0.56

VegetationLine0:	0.26

WaterMark0:	0.33

PopulationPresent0:	0.0

BeachToe1:	0.57

VegetationLine1:	0.41

WaterMark1:	0.7

PopulationPresent1:	0.0

BeachToe2:	0.7

VegetationLine2:	0.6

WaterMark2:	0.4

PopulationPresent2:	0.0

BeachToe3:	0.54

VegetationLine3:	0.88

WaterMark3:	0.28

PopulationPresent3:	nan

